In [3]:
import numpy as np
import pandas as pd
from copy import copy
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

import os
for dirname, _, filenames in os.walk('kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import warnings
warnings.filterwarnings('ignore')